## Model File Conversions

References:

https://www.tensorflow.org/lite/microcontrollers, https://blog.arduino.cc/2019/10/15/get-started-with-machine-learning-on-arduino/

In [1]:
# Do necessary imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import glob
list_keras_filenames = []

for file in glob.glob("*.h5"):
    list_keras_filenames.append(file)
    
print(list_keras_filenames)

['ANN_Ah_12.h5', 'ANN_Ah_22.h5', 'ANN_hybrid_12.h5', 'ANN_hybrid_22.h5', 'ANN_IV_12.h5', 'ANN_IV_22.h5', 'DNN_Ah_12.h5', 'DNN_Ah_22.h5', 'DNN_hybrid_12.h5', 'DNN_hybrid_22.h5', 'DNN_IV_12.h5', 'DNN_IV_22.h5']


In [3]:
for keras_filename in list_keras_filenames:
    keras_model = tf.keras.models.load_model(keras_filename)
    
    # Convert the model to the TensorFlow Lite format with float16 quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    model = converter.convert()
    
    # Save the tflite model to disk
    tflite_filename = keras_filename.strip(".h5") + ".tflite"
    open(tflite_filename, "wb").write(model)
    
    # hexdump the tflite file with name {tflite_name} into a cc header file, with the same name
    h_filename = keras_filename.strip(".h5") + ".h"
    !xxd -i $tflite_filename > $h_filename
    print("Conversion: {} >> {} >> {} done!".format(keras_filename, tflite_filename, h_filename))

Conversion: ANN_Ah_12.h5 >> ANN_Ah_12.tflite >> ANN_Ah_12.h done!
Conversion: ANN_Ah_22.h5 >> ANN_Ah_22.tflite >> ANN_Ah_22.h done!
Conversion: ANN_hybrid_12.h5 >> ANN_hybrid_12.tflite >> ANN_hybrid_12.h done!
Conversion: ANN_hybrid_22.h5 >> ANN_hybrid_22.tflite >> ANN_hybrid_22.h done!
Conversion: ANN_IV_12.h5 >> ANN_IV_12.tflite >> ANN_IV_12.h done!
Conversion: ANN_IV_22.h5 >> ANN_IV_22.tflite >> ANN_IV_22.h done!
Conversion: DNN_Ah_12.h5 >> DNN_Ah_12.tflite >> DNN_Ah_12.h done!
Conversion: DNN_Ah_22.h5 >> DNN_Ah_22.tflite >> DNN_Ah_22.h done!
Conversion: DNN_hybrid_12.h5 >> DNN_hybrid_12.tflite >> DNN_hybrid_12.h done!
Conversion: DNN_hybrid_22.h5 >> DNN_hybrid_22.tflite >> DNN_hybrid_22.h done!
Conversion: DNN_IV_12.h5 >> DNN_IV_12.tflite >> DNN_IV_12.h done!
Conversion: DNN_IV_22.h5 >> DNN_IV_22.tflite >> DNN_IV_22.h done!


Note: Write to a C file

The final step in preparing our model for use with TensorFlow Lite for Microcontrollers was to convert it into a C source file. You can see an example of this format in [`hello_world/sine_model_data.cc`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/hello_world/sine_model_data.cc). To do so, we can use a command line utility named [`xxd`](https://linux.die.net/man/1/xxd).